In [ ]:
# API_KEY = "PK13HT6JQJGE7IWZ9H10"
# SECRET_KEY = "0a90i4QBy0Z0cViVPYGAmuiziuyrIGsmgQIpqT6c"
# pip install alpaca-py pandas pytz

import os
import time
import pandas as pd
from alpaca.data import StockHistoricalDataClient, StockBarsRequest, TimeFrame
from alpaca.trading import TradingClient
from datetime import datetime, timedelta
import pytz
import logging
import threading

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# API 키 설정
API_KEY = "PK13HT6JQJGE7IWZ9H10" # API_KEY
API_SECRET = "0a90i4QBy0Z0cViVPYGAmuiziuyrIGsmgQIpqT6c" # API_SECRET

# 데이터 클라이언트 생성
data_client = StockHistoricalDataClient(API_KEY, API_SECRET)

# 거래 클라이언트 생성 (시장 시간 확인용)
trading_client = TradingClient(API_KEY, API_SECRET)

# 모니터링할 주식 심볼 목록
SYMBOLS = [
    # Basic Materials
    "VALE", "NEM", "FCX",
    # Communication Services
    "T", "NFLX", "WBD",
    # Consumer Cyclical
    "TSLA", "AMZN", "NKE",
    # Consumer Defensive
    "PEP", "KVUE", "PM",
    # Energy
    "PBR", "KMI", "SHEL",
    # Financial Services
    "ITUB", "USB", "MUFG",
    # Healthcare
    "AZN", "ABT", "UNH",
    # Industrials
    "CSX", "BA", "RTX",
    # Technology
    "NVDA", "ERIC", "YMM",
    # Utilities
    "NEE"
    ,"AAPL"
]

# 최근 데이터 저장용 딕셔너리 (심볼별 데이터 히스토리 유지)
# {symbol: [{timestamp, open, high, low, close, volume}, ...], ...}
historical_data = {symbol: [] for symbol in SYMBOLS}

def get_1min_bars_for_minute(target_minute):
    """특정 시점의 1분봉 데이터를 가져옵니다."""
    # 타겟 시간 (이전 1분간의 데이터)
    end = target_minute
    start = end - timedelta(minutes=1)
    
    # 디버깅용 로그
    logger.info(f"요청 시간 범위: {start} ~ {end} (뉴욕 시간)")
    
    # 요청 설정
    request_params = StockBarsRequest(
        symbol_or_symbols=SYMBOLS,
        timeframe=TimeFrame.Minute,
        start=start,
        end=end,
        adjustment='raw',
        feed='iex'  # 무료 피드 사용
    )

    try:
        # 데이터 요청
        bars = data_client.get_stock_bars(request_params)
        
        # 데이터프레임으로 변환
        df = bars.df
        
        if df is not None and not df.empty:
            bar_count = 0
            # 각 심볼별로 데이터 처리
            for symbol in SYMBOLS:
                try:
                    # 해당 심볼의 데이터만 필터링
                    symbol_data = df.xs(symbol, level=0) if symbol in df.index.get_level_values(0) else None
                    
                    if symbol_data is not None and not symbol_data.empty:
                        # 해당 기간의 봉 데이터 찾기
                        # 시간 필터링 - 타겟 시간에 가장 가까운 봉 선택
                        closest_bar = symbol_data.iloc[-1]
                        bar_time = closest_bar.name
                        
                        # 데이터 저장
                        bar_data = {
                            'timestamp': bar_time,
                            'open': closest_bar['open'],
                            'high': closest_bar['high'],
                            'low': closest_bar['low'],
                            'close': closest_bar['close'],
                            'volume': closest_bar['volume']
                        }
                        
                        # 히스토리에 데이터 추가
                        historical_data[symbol].append(bar_data)
                        
                        # 필요 시 히스토리 크기 제한 (예: 최근 100개만 유지)
                        if len(historical_data[symbol]) > 100:
                            historical_data[symbol] = historical_data[symbol][-100:]
                        
                        bar_count += 1
                    else:
                        logger.warning(f"{symbol}에 대한 데이터가 없습니다.")
                except KeyError:
                    logger.warning(f"{symbol}에 대한 데이터가 없습니다.")
                except Exception as e:
                    logger.error(f"{symbol} 데이터 처리 중 오류: {e}")
            
            logger.info(f"총 {bar_count}개 종목의 1분봉 데이터를 성공적으로 가져왔습니다.")
            return True
        else:
            logger.warning(f"해당 시간대({start} ~ {end})의 데이터가 없습니다. 시장이 열려있는지 확인하세요.")
            return False
    except Exception as e:
        logger.error(f"데이터 요청 중 오류 발생: {e}")
        return False

def check_market_open():
    """시장이 열려있는지 확인합니다."""
    try:
        clock = trading_client.get_clock()
        is_open = clock.is_open
        if is_open:
            next_close = clock.next_close.replace(tzinfo=pytz.UTC)
            next_close_local = next_close.astimezone(pytz.timezone('America/New_York'))
            logger.info(f"시장 열림. 마감 예정 시간: {next_close_local} (뉴욕 시간)")
        else:
            next_open = clock.next_open.replace(tzinfo=pytz.UTC)
            next_open_local = next_open.astimezone(pytz.timezone('America/New_York'))
            logger.info(f"시장 닫힘. 개장 예정 시간: {next_open_local} (뉴욕 시간)")
        return is_open
    except Exception as e:
        logger.error(f"시장 상태 확인 오류: {e}")
        return False

def print_latest_data():
    """가장 최근에 저장된 데이터를 출력합니다."""
    # 현재 시간 (뉴욕 시간)
    ny_time = datetime.now(pytz.timezone('America/New_York'))
    logger.info(f"현재 시간 (뉴욕): {ny_time}")
    logger.info("=" * 60)
    
    # 모든 티커 데이터 출력 (알파벳 순)
    for symbol in sorted(SYMBOLS):
        if historical_data[symbol] and len(historical_data[symbol]) > 0:
            latest = historical_data[symbol][-1]
            logger.info(f"{symbol:<5} - 시간: {latest['timestamp']}, 시가: ${latest['open']:.2f}, 고가: ${latest['high']:.2f}, 저가: ${latest['low']:.2f}, 종가: ${latest['close']:.2f}, 거래량: {latest['volume']}")
        else:
            logger.info(f"{symbol:<5} - 데이터 없음")
    
    logger.info("=" * 60)

def wait_until_next_minute():
    """다음 분의 정각(00초)까지 대기합니다."""
    now = datetime.now()
    
    # 현재 초가 55초 이상이면 다다음 분의 정각까지 대기 (API 지연 고려)
    if now.second >= 55:
        target = now + timedelta(minutes=2)
    else:
        target = now + timedelta(minutes=1)
    
    target = target.replace(second=0, microsecond=0)
    
    wait_seconds = (target - now).total_seconds()
    logger.info(f"다음 데이터 수집 시간({target.strftime('%H:%M:%S')})까지 {wait_seconds:.1f}초 대기...")
    
    return target, wait_seconds

def save_to_csv(minute_time):
    """특정 시점의 모든 종목 데이터를 CSV 파일로 저장합니다."""
    try:
        # 데이터를 DataFrame으로 변환
        rows = []
        for symbol in SYMBOLS:
            if historical_data[symbol] and len(historical_data[symbol]) > 0:
                # 최신 데이터
                data = historical_data[symbol][-1]
                
                row = {
                    'symbol': symbol,
                    'timestamp': data['timestamp'],
                    'open': data['open'],
                    'high': data['high'],
                    'low': data['low'],
                    'close': data['close'],
                    'volume': data['volume']
                }
                rows.append(row)
        
        if not rows:
            logger.warning("저장할 데이터가 없습니다.")
            return
            
        df = pd.DataFrame(rows)
        
        # 시간을 파일명에 포함
        time_str = minute_time.strftime('%Y%m%d_%H%M')
        filename = f"stock_data_{time_str}.csv"
        
        df.to_csv(filename, index=False)
        logger.info(f"데이터가 {filename}에 저장되었습니다.")
    except Exception as e:
        logger.error(f"CSV 저장 중 오류: {e}")

def main():
    logger.info(f"{len(SYMBOLS)}개 종목 실시간 1분봉 데이터 수집 시작...")
    logger.info("매 분 정각(00초)에 이전 1분간의 데이터를 수집합니다.")
    
    try:
        while True:
            is_market_open = check_market_open()
            
            if is_market_open:
                # 다음 분의 정각까지 대기
                target_time, wait_seconds = wait_until_next_minute()
                
                if wait_seconds > 0:
                    time.sleep(wait_seconds)
                
                # 뉴욕 시간대로 변환 (Alpaca API는 뉴욕 시간 기준)
                ny_timezone = pytz.timezone('America/New_York')
                target_time_ny = datetime.now().astimezone(ny_timezone).replace(second=0, microsecond=0)
                
                # 이전 1분간의 데이터 가져오기 (현재 시간 기준 이전 1분)
                previous_minute = target_time_ny - timedelta(minutes=1)
                
                logger.info(f"=== {target_time_ny.strftime('%H:%M')}에 {previous_minute.strftime('%H:%M')}의 1분봉 데이터 수집 중... ===")
                success = get_1min_bars_for_minute(target_time_ny)
                
                if success:
                    # 데이터 출력 및 저장
                    print_latest_data()
                    save_to_csv(target_time_ny)
                    
                # 데이터 수집 후 약간의 지연 (API 요청 간 간격 유지)
                time.sleep(1)
            else:
                logger.warning("현재 미국 주식 시장이 닫혀 있습니다.")
                # 15분마다 시장 상태 확인
                time.sleep(900)
            
    except KeyboardInterrupt:
        logger.info("프로그램을 종료합니다.")

if __name__ == "__main__":
    main()

2025-05-15 23:24:00,001 - INFO - === 10:24에 10:23의 1분봉 데이터 수집 중... ===
2025-05-15 23:24:00,001 - INFO - 요청 시간 범위: 2025-05-15 10:23:00-04:00 ~ 2025-05-15 10:24:00-04:00 (뉴욕 시간)
2025-05-15 23:24:00,866 - WARNING - VALE에 대한 데이터가 없습니다.
2025-05-15 23:24:00,870 - WARNING - NFLX에 대한 데이터가 없습니다.
2025-05-15 23:24:00,874 - WARNING - PBR에 대한 데이터가 없습니다.
2025-05-15 23:24:00,876 - WARNING - SHEL에 대한 데이터가 없습니다.
2025-05-15 23:24:00,877 - WARNING - USB에 대한 데이터가 없습니다.
2025-05-15 23:24:00,883 - WARNING - ERIC에 대한 데이터가 없습니다.
2025-05-15 23:24:00,885 - INFO - 총 23개 종목의 1분봉 데이터를 성공적으로 가져왔습니다.
2025-05-15 23:24:00,886 - INFO - 현재 시간 (뉴욕): 2025-05-15 10:24:00.886279-04:00
2025-05-15 23:24:00,887 - INFO - ============================================================
2025-05-15 23:24:00,888 - INFO - AAPL  - 시간: 2025-05-15 14:23:00+00:00, 시가: $210.18, 고가: $210.35, 저가: $210.17, 종가: $210.32, 거래량: 2259.0
2025-05-15 23:24:00,888 - INFO - ABT   - 시간: 2025-05-15 14:23:00+00:00, 시가: $132.06, 고가: $132.06, 저가: $131.98, 종가: $